In [1]:
# Following implementation is inspired by: http://gabrielelanaro.github.io/blog/2016/03/03/decision-trees.html
import numpy as np

In [2]:
# Training dataset
# x1 atmospheric pressure
x1 = ['high', 'high', 'low', 'low', 'low', 'high']  
# x2 is weather type
x2 = ['partly cloudy', 'sunny', 'sunny', 'cloudy', 'cloudy', 'cloudy']  
X = np.array([x1, x2]).T
y = np.array([False, False, True, False, False, True]) # rain (True) and no-rain (False)

In [3]:
X # features

array([['high', 'partly cloudy'],
       ['high', 'sunny'],
       ['low', 'sunny'],
       ['low', 'cloudy'],
       ['low', 'cloudy'],
       ['high', 'cloudy']], dtype='<U13')

In [4]:
y # labels

array([False, False,  True, False, False,  True])

In [5]:
# Splitting a set
# Input is an array of feature observations and output is a dictionary with "unique feature value" as key and indices as value
def partition(a):
    return {c: (a==c).nonzero()[0] for c in np.unique(a)}

# Picking which attribute to split
# Calculate entropy of a list
def entropy(s):
    res = 0
    val, counts = np.unique(s, return_counts=True)
    freqs = counts.astype('float')/len(s)
    for p in freqs:
        if p != 0.0:
            res -= p * np.log2(p)
    return res

# Calculate decrease in impurity (information gains)
# 
def mutual_information(y, x):
    
    # Calculate entropy of observation classes
    res = entropy(y)

    # We partition x, according to attribute values x_i
    val, counts = np.unique(x, return_counts=True)
    freqs = counts.astype('float')/len(x)

    # We calculate a weighted average of the entropy and subtract it from parent entropy
    for p, v in zip(freqs, val):
        res -= p * entropy(y[x == v])

    return res

# Checks for pureness of elements in a list
def is_pure(s):
    return len(set(s)) == 1

# Helper function to print decision tree
def print_tree(d, depth = 0):
    for key, value in d.items():
        for i in range(depth):
                print(' ', end='')
        if type(value) is dict:
            print(key, end=':\n')
            print_tree(value, depth + 1)
        else:
            print(key, end=': ')
            print(value)
            
# Get the most common element of an array
def most_common(a):
    (values,counts) = np.unique(a,return_counts=True)
    ind=np.argmax(counts)
    return values[ind]

# Recursive split of observations
def recursive_split(x, y):
    
    # If set to be split is pure or empty, return it as leaf
    if is_pure(y) or len(y) == 0:
        return most_common(y)

    # Calculate decrease in impurity (information gain) and split attribute with maximum gain
    gain = np.array([mutual_information(y, x_attr) for x_attr in x.T])
    selected_attr = np.argmax(gain)

    # Sufficiently pure, return it as leaf
    if np.all(gain < 1e-6):
        return most_common(y)

    # Split using the selected attribute
    sets = partition(x[:, selected_attr])

    # Perform recursive splits and collect results
    res = {}
    for key, value in sets.items():
        y_subset = y.take(value, axis=0)
        x_subset = x.take(value, axis=0)
        res["x_%d = %s" % (selected_attr, key)] = recursive_split(x_subset, y_subset)

    return res

# Perform algorithm on the example dataset to create a decision tree
d = recursive_split(X, y)
print_tree(d)

x_1 = cloudy:
 x_0 = high: True
 x_0 = low: False
x_1 = partly cloudy: False
x_1 = sunny:
 x_0 = high: False
 x_0 = low: True


In [6]:
# New dataset (which shall be classified by the above generated decision tree)
x1 = ['high', 'low', 'low', 'high', 'low', 'high', 'high', 'low', 'low', 'high', 'low', 'low']
x2 = ['sunny', 'sunny', 'cloudy', 'cloudy', 'partly cloudy', 'cloudy', 'partly cloudy', 'cloudy', 'sunny', 'cloudy', 'cloudy', 'partly cloudy']
X = np.array([x1, x2]).T
y = np.array([False, True, True, False, False, True, False, True, True, False, True, True]) # ground-truth of classification

In [7]:
# 1.2) Predicting the classes of new Dataset based on splitting algoithm used in tree D
import pandas as pd

#Function for prediction of class which takes input as numpy array of test dataset and returns predicted classes
def predictClass(X):
    predicted = []
    for X0,X1 in X:
        key = "x_1 = "+X1
        if key in d.keys():
            if type(d[key]) is dict:
                X0_dict = d[key]
                key = "x_0 = "+X0
                predicted.append(X0_dict[key])
            else:
                predicted.append(d[key])
                
    predicted = np.array(predicted)
    return predicted

predicted = predictClass(X) ##Calling function predictClass.

wether = {'atmospheric pressure':x1, 'weather type':x2, 'Predicted Class':predicted, 'Ground truth': y}
wetherDF = pd.DataFrame(wether)
wetherDF.style

,atmospheric pressure,weather type,Predicted Class,Ground truth
0,high,sunny,False,False
1,low,sunny,True,True
2,low,cloudy,False,True
3,high,cloudy,True,False
4,low,partly cloudy,False,False
5,high,cloudy,True,True
6,high,partly cloudy,False,False
7,low,cloudy,False,True
8,low,sunny,True,True
9,high,cloudy,True,False


In [8]:
#1.3) Calculating the overall Error rate

error = 0
for p,actual in zip(predicted,y):
    if p!=actual:
        error+=1
errorRate = error/len(y)
print("The overall error rate is " , errorRate*100, "%")

The overall error rate is  50.0 %
